In [5]:
source('R_functions/cdr3-QTL_functions.R')
source('R_functions//hla_functions.R')
source('libraries.R')


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘tidyr’


The following object is masked from ‘package:magrittr’:

    extract


Registered S3 methods overwritten by 'ggtern':
  method           from   
  grid.draw.ggplot ggplot2
  plot.ggplot      ggplot2
  print.ggplot     ggplot2

--
Remember to cite, run citation(package = 'ggtern') for further info.
--


Attaching package: ‘ggtern’


The following objects are masked from ‘package:ggplot2’:

    aes, annotate, ggplot, ggplot_build, ggplot_gtable, ggplotGrob,
    ggsave, layer_data, theme_bw, theme_classic, theme_dark,
    theme_gray, theme_light, theme_linedraw, theme_minimal, theme_void


Loading required package: viridisLite


Attaching package: ‘arrow’


The following obje

### Making matrices for MANOVA and Conditional in DOWNSAMPLED CDR3

In [25]:
hla_genotypes_matrix_wo_low_ab <- fread('../data/hla/hla_genotypes_matrix_wo_low_ab.csv')
pca_hla <- fread('../data/hla/pca_hla_9PCs.tsv')
hla_matrices <- paste0('../hla_matrices/with_pcs/',list.files('../hla_matrices/with_pcs/'))
pcs <- 3
dir_downsampling <- '../CDR3_downsampling_experiment/unique/'
dir_content <- paste0(dir_downsampling, list.files(dir_downsampling))

for (path_in in dir_content[2:length(dir_content)]){
    sample_size <- unlist(strsplit(path_in, '/'))[4]
    source("Make_matrices_for_Main_Manova.R", chdir = TRUE)
    
}


In [ ]:
path_in <- "/work_beegfs/sukmb667/projects/cdr3-qtl/healthy_and_ibd/CDR3_downsampling_experiment/unique/"
directories <- list.files(path_in, full.names = TRUE)
pairs_dir <- directories[file.info(directories)$isdir]

for (tsv_dir in pairs_dir) {
    sample_size <- strsplit(tsv_dir, "/")[[1]][length(strsplit(tsv_dir, "/")[[1]])]
    # List all .tsv files in the directory
    tsv_files <- list.files(paste0(tsv_dir, '/manova_results/cdr3_hla_pairs/irt_freq_unique/'), pattern = "\\.tsv$", full.names = TRUE)
 
    # Read all .tsv files into a list of data.tables
    data_table_list <- lapply(tsv_files, fread)

    save(data_table_list, file = paste0(tsv_dir,'/',sample_size,"_irt_freq_unique.RData"))
    
}

### Main Manova on CDR3 Downsampled

In [ ]:
source('/work_beegfs/sukmb667/projects/cdr3-qtl/healthy_and_ibd/scripts/Main_manova_from_matrices.R', chdir = TRUE)

#### Export .db to the .csv

In [36]:
db_path <- "/work_beegfs/sukmb667/projects/cdr3-qtl/healthy_and_ibd/CDR3_downsampling_experiment/unique/cdr3-downsampling-unique.db"
conn <- dbConnect(RSQLite::SQLite(), db_path)
tables <- dbListTables(conn)
df <- dbReadTable(conn,  tables)

output_path <- "/work_beegfs/sukmb667/projects/cdr3-qtl/healthy_and_ibd/CDR3_downsampling_experiment/unique/Downsampling_manova_all.csv"
fwrite(df, output_path, row.names = FALSE)

dbDisconnect(conn)

In [ ]:
first <- TRUE
sample_size <- '100000'
path_pairs <- paste0(path_in, '/manova_results/cdr3_hla_pairs/irt_freq_unique/')
pairs_to_test <- paste0(path_pairs, list.files(path_pairs))
#pairs_to_test <- grep(paste(tested_pairs, collapse = "|"), list.files(path_pairs), value = TRUE, invert = TRUE, fixed = TRUE)
for (pair in pairs_to_test){
    pair_matrix <- fread(pair)
    dir_results <- paste0(path_in, '/manova_results/irt_freq_unique/')
    manova_df <- mlm_fun(pair_matrix, dir_results, n_pcs = pcs) %>% 
        separate(pair, into = c('HLA', 'Site_hla', 'Length_cdr3', 'Position_cdr3'), sep = '_', remove = FALSE)
    if (first){
        manova_df_all <- manova_df[0,]
        first <- FALSE
    } else {
    manova_df_all <- rbind(manova_df_all, manova_df) 
        }
}

### Conditional on Downsampled CDR3

In [ ]:
hla_matrices <- paste0('../hla_matrices/with_pcs/',list.files('../hla_matrices/with_pcs/'))
pcs <- 3

for (path_in in dir_content{
    dir_analysis <- paste0(path_in, '/conditional_analysis/')
    if (!file.exists(dir_analysis)) {
        dir.create(dir_analysis, recursive = TRUE)
        }
    pairs_to_test <- paste0(path_in,'/cdr3_hla_pairs/')
    sample_size <- unlist(strsplit(path_in, '/'))[3]
    manova_cond_all <- fread(paste0(path_in,'/',sample_size,'_manova.tsv'))
    bonf <- 0.01 / nrow(na.omit(manova_cond_all))
            
    source('Conditional_analysis_updated.R')
                 }